In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [ ]:
import glob
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from itertools import combinations, permutations, product

from math import sqrt
from pmdarima import auto_arima
from src.utils import util as utl
from sklearn.metrics import mean_absolute_error, r2_score


global companies_set 

warnings.filterwarnings('ignore')

# 1 - Análise dos mercados (independente da empresa)

In [ ]:
door_dir = glob.glob(r"D:\Documentos_D\UFCG\2021.2e\TCC\Project\analise_de_indicador_tcc_ufcg\data\processed\**\*" + ".csv", recursive=True)

df = pd.DataFrame()

for file_path in door_dir:
    temp_df = pd.read_csv(file_path)
    df = df.append(temp_df)

df = df.reset_index().drop(['index'], axis=1)
df['Volume_Venda'] = df['Volume_Venda'].round(2)


In [ ]:
dfm = pd.DataFrame()
temp_df = df[((df.Mercado == 'AN') & (df.Empresa == 'E2'))].reset_index().drop(['index'],axis=1)

dfm['Data'] = temp_df.Data
dfm['Data'] = pd.to_datetime(dfm.Data)

# ----------------------------------------------------AN:
r1_bc = df[((df.Mercado == 'AN') & (df.Empresa == 'E1'))].Volume_Venda.astype(float).to_frame()
r1_br = df[((df.Mercado == 'AN') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

dfm['AN'] = r1_bc.add(r1_br,axis=0)
# ----------------------------------------------------AS:
r2_bi = df[((df.Mercado == 'AS') & (df.Empresa == 'E3'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r2_br = df[((df.Mercado == 'AS') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

r2_bt_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(18):
    values.append(0)
r2_bt_temp['Volume_Venda'] = values
r2_bt = df[((df.Mercado == 'AS') & (df.Empresa == 'E4'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r2_bt = r2_bt.append(r2_bt_temp).reset_index().drop(['index'], axis=1)

dfm['AS'] = r2_bi.add(r2_br,axis=0)
dfm['AS'] = dfm['AS'] + r2_bt['Volume_Venda']
# ----------------------------------------------------EU:
r3_bi = df[((df.Mercado == 'EU') & (df.Empresa == 'E3'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r3_br = df[((df.Mercado == 'EU') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

dfm['EU'] = r3_bi.add(r3_br,axis=0)
# ----------------------------------------------------AL:
r4_br_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(13):
    values.append(0)
r4_br_temp['Volume_Venda'] = values
r4_br = df[((df.Mercado == 'AL') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r4_br = r4_br.append(r4_br_temp).reset_index().drop(['index'], axis=1)

r4_bt_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(18):
    values.append(0)
r4_bt_temp['Volume_Venda'] = values
r4_bt = df[((df.Mercado == 'AL') & (df.Empresa == 'E4'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r4_bt = r4_bt.append(r4_bt_temp).reset_index().drop(['index'], axis=1)

dfm['AL'] = r4_br.add(r4_bt,axis=0)
# ----------------------------------------------------JP:
r5_bi_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(56):
    values.append(0)
r5_bi_temp ['Volume_Venda'] = values
r5_bi = df[((df.Mercado == 'JP') & (df.Empresa == 'E3'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r5_bi = r5_bi_temp.append(r5_bi).reset_index().drop(['index'], axis=1)

r5_br = df[((df.Mercado == 'JP') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

r5_bt_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(18):
    values.append(0)
r5_bt_temp ['Volume_Venda'] = values
r5_bt = df[((df.Mercado == 'JP') & (df.Empresa == 'E4'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r5_bt = r5_bt.append(r5_bt_temp ).reset_index().drop(['index'], axis=1)

dfm['JP'] = r5_bi.add(r5_br, axis=0)
dfm['JP'] = dfm['JP'] + r5_bt['Volume_Venda']
# ----------------------------------------------------BRL:
r6_br = df[((df.Mercado == 'BRL') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
dfm['BRL'] = r6_br

# ----------------------------------------------------Indefinido:
indefinido = df[((df.Mercado == 'Indefinido'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
dfm['Indefinido'] = indefinido

dfm = dfm.set_index('Data')

display(dfm.head(5))

In [ ]:
markets_names = ['AN','AS','EU','AL','JP','BRL'] # O mercado 'Indefinido' será desconsiderado;
markets_values = []
for market_name in markets_names:  
    values_market_df = dfm[market_name]
    markets_values.append(values_market_df.to_frame())

In [ ]:
# Criando  volume de dados para treinamento do mercado da América Latina ('AL'):
date_indexes = []
sales_values = []

for market_values_df, market_name in zip(markets_values, markets_names):
    if market_name in ['AL']:
        temp_df =  market_values_df
        for i, row in temp_df.iterrows():
            date = i
            value = row['AL']

            if date < pd.Timestamp('2016-01-01'): # Coletando valores até 01/12/2015;
                date_indexes.append(i)
                sales_values.append(value)

df_AL = pd.DataFrame()
df_AL['Data'] =  date_indexes
df_AL['AL'] =  sales_values
df_AL = df_AL.set_index('Data')
markets_values[3] = df_AL

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.plotting import lag_plot, autocorrelation_plot
from statsmodels.tsa.stattools import adfuller

p_values = []

#Verificando estacionariedade com adfuller:
for market_values_df, market_name in zip(markets_values, markets_names):
    series = market_values_df
    X = series.values
    train, test = X[1:len(X)-12], X[len(X)-12:]
    result = adfuller(train, autolag='AIC')
    p_values.append((market_name,np.round(result[0],4),np.round(result[1],4)))

print(p_values)

In [ ]:
# Utilizando a bilbioteca autoARIMA para verificar a sugestão de melhor modelo de ARIMA para os seis mercados:

for market_values_df, market_name in zip(markets_values, markets_names):
    print("Mercado: ", market_name)
    X = market_values_df.values
    train, test = X[1:len(X)-12], X[len(X)-12:]
    model = auto_arima(train, start_P=0, start_q=0, stepwise=True, trace=True)

In [ ]:
windows_lags = []
for wg in product([1,2,3,4,5,6,7,8,9,10,11,12], repeat = 2):
    windows_lags.append(wg)

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

maes_autoreg = []
r2s_autoreg = []

results_autoreg ={}

for market_values_df, market_name in zip(markets_values, markets_names): 
    # Separando o dataset entre treino e teste:
    series = market_values_df
    X = series.values
    train, test = X[1:len(X)-12], X[len(X)-12:]

    wgs_autoreg = []
    for wg in windows_lags:
        # Treinando a autoregressão:
        window = wg[0]
        lags = wg[1]

        if lags >= window:
            model = AutoReg(train, lags=lags)
            model_fit = model.fit()
            coef = model_fit.params

            # Caminhando sobre o dataset de teste para comparar o valor real com o predito:
            history = train[len(train)-window:]
            history = [history[i] for i in range(len(history))]
            predictions = []

            try: 
                for t in range(len(test)):
                    length = len(history)
                    lag = [history[i] for i in range(length-window,length)]
                    yhat = coef[0]

                    for d in range(window):
                        yhat += coef[d+1] * lag[window-d-1]

                    obs = test[t]
                    predictions.append(yhat)
                    history.append(obs)
                    # print('predicted=%f, expected=%f' % (yhat, obs))
            except:
                print('Window: ', window)
                print('Lags: ', lags )

            mae = mean_absolute_error(test, predictions)
            maes_autoreg.append(mae)

            r2_scr = r2_score(test, predictions)
            r2s_autoreg.append(r2_scr)

            wgs_autoreg.append((wg, r2_scr, mae))

            # print("r²: ", r2_scr)
            # print("mae: ", mae)
                    
            # # Plotando dados:
            # plt.plot(test)
            # plt.xticks(ticks=[0,1,2,3,4,5,6,7,8,9,10,11],labels=[1,2,3,4,5,6,7,8,9,10,11,12])
            # plt.title("Autoregressão Simples - " + market_name)
            # plt.xlabel("Mês")
            # plt.ylabel("Volume de Venda (t)")
            # plt.plot(predictions, color='red')
            # plt.show()
    results_autoreg[market_name] = wgs_autoreg 

In [ ]:
res_autoreg = []
for key in results_autoreg.keys():
    temp_order = []
    temp_r2 = []
    temp_mae = []
    for tpl in results_autoreg[key]:
        order = tpl[0]
        r2 = tpl[1]
        mae = tpl[2]

        temp_order.append(order)
        temp_r2.append(r2)
        temp_mae.append(mae)

    max_pos = temp_r2.index(max(temp_r2))
    res_autoreg.append((key, temp_order[max_pos], temp_r2[max_pos], temp_mae[max_pos]))

print(res_autoreg)

In [ ]:
orders = []
for ord in product([0,1,2,3,4,5,6,7,8,9,10,11,12], repeat = 3):
    if ord[1] == 0:
        orders.append(ord)

print(orders)
# comb = list(permutations([0,1,2,3,4,5,6,7,8,9,10,11,12],3))
# orders = []
# for p in comb:
#     if p[1] == 0:
#         orders.append(p)

# print(orders)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# # Criação de lista com permutações, para teste de ordens (p,d,q) no ARIMA:
# comb = list(permutations([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],3))
# size = len(comb)
# orders = []
# for i in range(len(comb)):
#     if comb[i][1] == 0:
#         orders.append(comb[i])

orders = [(1,2,1)]
maes_arima = []
r2s_arima = []

i = 0

results_arima = {}
for market_values_df, market_name in zip(markets_values, markets_names):
    if market_name != 'AL':
    
        X = market_values_df[market_name]
        train, test = X[1:len(X)-12], X[len(X)-12:]

        ords_arima = []
        for ord in orders:
            model = ARIMA(train, order=ord)
            
            try:
                res = model.fit()
                predictions = res.predict(start=pd.to_datetime('2021-01-01'), end=pd.to_datetime('2021-12-01'), dynamic=False)
            except:
                pass

            mae = mean_absolute_error(test, predictions)
            maes_arima.append(mae)

            r2_scr = r2_score(test, predictions)
            r2s_arima.append(r2_scr)

            ords_arima.append((ord, r2_scr, mae))
        
        print(market_name+" finalizado!")

        results_arima[market_name] = ords_arima 
    

In [ ]:
res_arima = []
for key in results_arima.keys():
    temp_order = []
    temp_r2 = []
    temp_mae = []
    for tpl in results_arima[key]:
        order = tpl[0]
        r2 = tpl[1]
        mae = tpl[2]

        temp_order.append(order)
        temp_r2.append(r2)
        temp_mae.append(mae)

    max_pos = temp_r2.index(max(temp_r2))
    res_arima.append((key, temp_order[max_pos], temp_r2[max_pos], temp_mae[max_pos]))

print(res_arima)
 

In [ ]:
for market_values_df, market_name in zip(markets_values, markets_names):
    plot_acf(market_values_df[market_name], lags=12)
    plt.title("AutoCorrelação - "+ market_name)
    plt.xticks(ticks=[0,1,2,3,4,5,6,7,8,9,10,11],labels=[1,2,3,4,5,6,7,8,9,10,11,12])
    plt.show()

    plot_pacf(market_values_df[market_name], lags=12)
    plt.title("AutoCorrelação Parcial - "+ market_name)
    plt.xticks(ticks=[0,1,2,3,4,5,6,7,8,9,10,11],labels=[1,2,3,4,5,6,7,8,9,10,11,12])
    plt.show()

In [28]:
# Utilizando XGBoosting
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot

# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]
 
# fit an xgboost model and make a one step prediction
def xgboost_forecast(train, testX):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict(asarray([testX]))
	return yhat[0]
 
# walk-forward validation for univariate data
def walk_forward_validation(train, test, n_test):
	predictions = list()
	# split dataset
	# train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction
		yhat = xgboost_forecast(history, testX)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		# summarize progress
		print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, -1], predictions

In [30]:

for market_values_df, market_name in zip(markets_values, markets_names):
    X = market_values_df[market_name]
    train, test = X[1:len(X)-12], X[len(X)-12:]

    # load the dataset
    values = train.values
    # transform the time series data into supervised learning
    train = series_to_supervised(train, n_in=6)
    test = series_to_supervised(test, n_in=6)
    # evaluate
    mae, y, yhat = walk_forward_validation(train, test, 12)
    print('MAE: %.3f' % mae)
    # plot expected vs predicted
    pyplot.plot(y, label='Expected')
    pyplot.plot(yhat, label='Predicted')
    pyplot.legend()
    pyplot.show()
    

IndexError: tuple index out of range